# League of Legends Role Models: A Recommendation System for Skill Training and Study

__Gwen Rathgeber | [GitHub](https://github.com/gwenrathgeber) | [LinkedIn](https://www.linkedin.com/in/gwenrathgeber/)__  


_Special thanks to [Uthgar](https://twitter.com/Dr_Uthgar) at Mobalytics.gg for suggesting this project_ 


_The accompanying Flask app to this project can be viewed at https://github.com/gwenrathgeber/lol_role_models_app_

## Problem Statement

League of Legends is a Multiplayer Online Battle Arena game known for attracting devoted players with a strong competitive drive. As one of the earliest games to popularize online video game streaming on platforms such as Twitch.tv, League demonstrated the appeal of not only playing games, but watching others compete. Due to the complexity of the game, a highly effective method of improving is studying the gameplay of higher-skilled players. We wish to improve this process for players by providing a recommendation system which matches players with Role Models: high-ranking players whose playstyles are similar to the user. Users can then use other tools to watch replays of recent games played by their recommended Role Models, simplifying and enhancing the process of finding high-level games to study through personalization and filtering.

## Executive Summary
In order to implement the recommender, we require a body of high-level players who will be recommended to users. All necessary data will be acquired from the Riot Games API, which can provide match-level data for games played by on a given account.

After scraping the data we need, we will need to combine each players' match history into a single row of aggregate statistics, using subject-matter knowledge to identify metrics which will be represent distinctive play patterns while also being comparable between lower- and higher-skilled players.

Finally, we will implement the recommendation system itself, which will involve pulling an individual player's match history, performing our aggregation, and comparing their statistics against our Role Model candidates.

Evaluation of this recommendation system is unfortunately limited to subjective interpretation of results by myself and test users, as it is not designed to drive a particular business objective or KPI. However, if a similar tool was deployed on one of the many stat-tracking and personal improvement websites which exist for League of Legends Players, some important target metrics would include the page bounce rate for the tool, churn and retention rates of tool users vs. non-users, and user feedback in the form of qualitative and quantitative satisfaction surveys that could be added to the results page. 

### Table of Contents
- [Problem Statement](#Problem-Statement)
- [Executive Summary](#Executive-Summary)
- [Data Dictionary](#Data-Dictionary)
- [API Scraping](#API-Scraping)
- [Data Processing](#Data-Processing)
- [Recommender](#Recommender)
- [Future Work](#Future-Work)
- [References](#References)

## Data Dictionary

The following endpoints of the Riot API were used over the course of this project, please consult their documentation for details of the fields:
 - [Summoner by Account ID Endpoint](https://developer.riotgames.com/apis#summoner-v4/GET_getByAccountId)
 - [Summoner by Account Name Endpoint](https://developer.riotgames.com/apis#summoner-v4/GET_getBySummonerName)
 - [Challenger Leagues Endpoint](https://developer.riotgames.com/apis#league-v4/GET_getChallengerLeague)
 - [Match Histories Endpoint](https://developer.riotgames.com/apis#match-v4/GET_getMatchlist)
 - [Match Data Endpoint](https://developer.riotgames.com/apis#match-v4/GET_getMatch)
 - [Timeline Data Endpoint](https://developer.riotgames.com/apis#match-v4/GET_getMatchTimeline)

The following is a representation of the custom fields we generated in our game data analysis:


Column | Data Type| Description
-|-|-
csd_10 | float|Average CS (Creep Score) Difference Between Player and Opponent at 10 minutes
gold_d_10 | float| Average Gold Difference Between Player and Opponent at 10 minutes
xpd_10| float|Average Experience Difference Between Player and Opponent at 10 minutes
dmg_share| float| Average Percent of Team's Total Damage to Champions Dealt by Player
dmg_taken_share| float| Average Percent of Team's Total Damage Taken Received by Player
vision_score| float| Average Vision Score of Player
kill_participation|float|Average Percent of Team's Kills a Player was Involved In
obj_dmg_share|float| Average Percent of Team's Total Damage to Epic Monsters and Structures by Player
dragons|float|Average Percent of Dragons Taken by Player's Team
barons|float|Average Percent of Barons Taken by Player's Team
wards_cleared |float | Average Ratio of Wards Cleared to Wards Placed by Player
vision_wards_purchased |float | Average Ratio of Vision Wards Purchased to Wards Placed by Player
kda_early | float| Average Ratio of Kills + Deaths + Assists Occurring before 10 minutes
kda_mid |float | Average Ratio of Kills + Deaths + Assists Occurring between 10 and 20 minutes
kda_late |float| Average Ratio of Kills + Deaths + Assists Occurring after 20 minutes
solo_kills |float | Average Ratio of Kills with no Assists by Teammates
teamfight_kills |float | Average Ratio of Kills with two or more Assists by Teammates
skirmish_kills |float | Average Ratio of Kills with only one Assist by Teammates
wards_early |float | Average Ratio of Wards Placed before 10 minutes
wards_mid |float |  Average Ratio of Wards Placed between 10 and 20 minutes
wards_late |float |  Average Ratio of Wards Placed after 20 minutes
most_played_champ |string | Most Frequently Played Champion
most_played_role |string | Most Frequently Played Role
most_played_lane | string| Most Frequently Played Lane
role |string | Simplified Most Played Role
op_gg | string| URL of player's op.gg account profile
similarity|float|Euclidean Distance from Role Model Player to User


In [2]:
# Imports and constants
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import jsonlines
import json
import time
from sklearn.preprocessing import StandardScaler
import requests
import concurrent.futures

region_base_url_dict = {'na':'na1.api.riotgames.com',
          'br':'br1.api.riotgames.com',
          'eun':'eun1.api.riotgames.com',
          'euw':'euw1.api.riotgames.com',
          'jp':'jp1.api.riotgames.com',
          'kr':'kr.api.riotgames.com',
          'la1':'la1.api.riotgames.com',
          'la2':'la2.api.riotgames.com',
          'oce':'oc1.api.riotgames.com',
          'tr':'tr1.api.riotgames.com',
          'ru':'ru.api.riotgames.com'}
            
api_key = pd.read_json('../secrets.json')['riot_api_key'][0]

timeline_by_match_id_url = '/lol/match/v4/timelines/by-match/'

match_by_match_id_url = '/lol/match/v4/matches/'

account_by_name_url = '/lol/summoner/v4/summoners/by-name/'

match_hist_by_id_url = '/lol/match/v4/matchlists/by-account/'

challenger_ladder_url = '/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'

summoner_by_summoner_id = '/lol/summoner/v4/summoners/'

champions = requests.get('http://ddragon.leagueoflegends.com/cdn/10.16.1/data/en_US/champion.json').json()

champions_df = pd.DataFrame(champions['data']).transpose()

## API Scraping

Scraping the Riot API had some unique challenges. The rate limit for a developer key (the basic level of access) is low, at 100 requests per 2 minutes. To get a large enough pool of players to recommend as Role Models, we split up our requests by the region of each player, as each regional endpoint has its own rate limit. First, we pulled a list of all players in the Challenger division (top ~200 players per server), then used multithreading to request all their matches, with each thread querying a different region.

In [ ]:
def get_all_challengers():
    challengers_df = pd.DataFrame()
    for key, base_url in region_base_url_dict.items():
        ladder_response = requests.get(f'https://{base_url}{challenger_ladder_url}?api_key={api_key}')
        try:
            assert(ladder_response.status_code == 200)
            response_df = pd.DataFrame(ladder_response.json()['entries'])
            response_df['region'] = [key] * len(response_df)
            challengers_df = pd.concat([challengers_df,response_df])
        except:
            print(f'Bad request for {key}: {ladder_response.status_code}')
        time.sleep(1.2001)
        
    return challengers_df

In [8]:
def get_summoner_by_name(base_url, summoner_name):
    return requests.get(f'https://{base_url}{account_by_name_url}{summoner_name}?api_key={api_key}')

In [4]:
def get_summoner(base_url, summoner_id):
    return requests.get(f'https://{base_url}{summoner_by_summoner_id}{summoner_id}?api_key={api_key}')

In [5]:
def get_match_hist(account_id, base_url, queue = '420'):
    return requests.get(f'https://{base_url}{match_hist_by_id_url}{account_id}?api_key={api_key}&queue={queue}')

In [6]:
def get_match(match_id, base_url):
    return requests.get(f'https://{base_url}{match_by_match_id_url}{match_id}?api_key={api_key}'),requests.get(f'https://{base_url}{timeline_by_match_id_url}{match_id}?api_key={api_key}')

In [ ]:
def get_challenger_match_hists(region, challengers_df = all_challengers, has_account_id=False):
    challengers_df = challengers_df[['summonerId','region']]
    challengers_df['player_ids'] = [np.nan] * len(challengers_df)
    challengers_df = challengers_df[challengers_df['region'] == region]
    challengers_df.reset_index(drop=True,inplace=True)
    
    if has_account_id==False:
        for i, tuples in enumerate(challengers_df.values):
            if i % 500 == 0:
                print(f'{i} of {len(challengers_df)} account ids')
            summoner_id, region, unused_pid = tuples
            summoner = get_summoner(region_base_url_dict[region],summoner_id).json()
            account_id = summoner['accountId']
            challengers_df.loc[i,'account_ids'] = account_id
            all_challengers.loc[i,'account_ids'] = account_id
            time.sleep(1.2)
        
    challenger_match_hists = pd.DataFrame()
    for i, account_id in enumerate(challengers_df['account_ids']):
        if i % 500 == 0:
            print(f'{i} of {len(challengers_df)} match histories')
        try:
            match_hist = pd.DataFrame(get_match_hist(account_id, region_base_url_dict[challengers_df.loc[i,'region']]).json()['matches'])
            match_hist['region'] = [challengers_df.loc[i,'region']] * len(match_hist)
            match_hist['account_id'] = [account_id] * len(match_hist)
            challenger_match_hists = pd.concat([challenger_match_hists, match_hist])
        except:
            pass
        time.sleep(1.2)
    
    return challenger_match_hists

In [ ]:
def scrape_seeds(region):
    print(f'{region} thread initialized')
    start_time = time.time()
    base_url = region_base_url_dict[region]
    
    matches_list = []
    scraped_ids = set()
    timelines_list = []
    scraped_participants = set()
    unscraped_ids = set()
    scraped_matches = []
    
    with jsonlines.open(f'../data/scraped_ids_{region}.jsonl') as infile:
        print(f'reading scraped_ids_{region}')
        for line in infile.iter():
            scraped_ids.add(line)       
    
    with jsonlines.open(f'../data/unscraped_ids_{region}.jsonl') as infile:
            print(f'reading unscraped_ids_{region}')
            for line in infile.iter():
                unscraped_ids.add(line)
                
    with jsonlines.open(f'../data/matches_{region}.jsonl') as infile:
            print(f'reading matches_{region}')
            for line in infile.iter():
                scraped_matches.append(dict(line)['gameId'])     
    
    seed = pd.read_csv('../data/challenger_match_hists_ranked_only.csv')
    
    seed = seed[seed['region']==region]
    
    seed.reset_index(drop=True,inplace=True)
    
    for i, account_id in enumerate(seed['account_id']):
        if account_id not in scraped_ids or seed.loc[i,'gameId'] not in scraped_matches:
            scraped_ids.add(account_id)
            match, timeline = get_match(seed.loc[i,'gameId'], base_url)
        
            if match.status_code == 200 and timeline.status_code == 200:
                matches_list.append(match.json())
                timelines_list.append(timeline.json())
                [scraped_participants.add(part['player']['accountId']) for part in match.json()['participantIdentities']]
                scraped_matches.append(seed.loc[i,'gameId'])
            else:
                print(f'matches error: {match.status_code}\ntimelines error: {timeline.status_code}\nSummoner: {account_id}\nRegion:{region}')
            time.sleep(2.4)
        
        else:
            pass
        
        if 'match' in locals():
            if i % 100 == 0 or match.status_code == 403:
                print(f'{i} matches scraped of {len(seed)}\nRegion: {region}\n')
                try:
                    with open(f'../data/matches_{region}.jsonl', 'a') as outfile:
                        for entry in matches_list:
                            json.dump(entry, outfile)
                            outfile.write('\n')
                            matches_list = []

                    with open(f'../data/timelines_{region}.jsonl', 'a') as outfile:
                        for entry in timelines_list:
                            json.dump(entry, outfile)
                            outfile.write('\n')
                            timelines_list = []

                    with open(f'../data/scraped_ids_{region}.jsonl', 'w') as outfile:
                        for entry in scraped_ids:
                            json.dump(entry, outfile)
                            outfile.write('\n')

                    unscraped_ids = unscraped_ids.union(scraped_participants)
                    unscraped_ids -= scraped_ids

                    with open(f'../data/unscraped_ids_{region}.jsonl', 'w') as outfile:
                        for entry in unscraped_ids:
                            json.dump(entry, outfile)
                            outfile.write('\n')
                except:
                    pass

                if match.status_code == 403:
                    return None
            
    with open(f'../data/matches_{region}.jsonl', 'a') as outfile:
        for entry in matches_list:
            json.dump(entry, outfile)
            outfile.write('\n')

    with open(f'../data/timelines_{region}.jsonl', 'a') as outfile:
        for entry in timelines_list:
            json.dump(entry, outfile)
            outfile.write('\n')

    with open(f'../data/scraped_ids_{region}.jsonl', 'a') as outfile:
        for entry in scraped_ids:
            json.dump(entry, outfile)
            outfile.write('\n')

    print(f'Seeds scraped, region = {region}')

    unscraped_ids = unscraped_ids.union(scraped_participants)
    unscraped_ids -= scraped_ids
    
    with open(f'../data/unscraped_ids_{region}.jsonl', 'w') as outfile:
        for entry in unscraped_ids:
            json.dump(entry, outfile)
            outfile.write('\n')
    

In [8]:
def remove_duplicates(region):
    match_list = []
    timeline_list = []
    
    with jsonlines.open(f'../data/matches_{region}.jsonl') as infile:
                for line in infile.iter():
                    match_list.append(line) 
                    
    with jsonlines.open(f'../data/timelines_{region}.jsonl') as infile:
            for line in infile.iter():
                    timeline_list.append(line)
    
    del infile
    ranked_matches = []
    ranked_timelines = []
    is_ranked = []
    game_ids = set()
    
    for i, match in enumerate(match_list):
        if match['queueId'] == 420 and match['gameId'] not in game_ids:
            ranked_matches.append(match)
            is_ranked.append(i)
            game_ids.add(match['gameId'])
    del match_list
    
    
    with open(f'../data/cleaned/matches_{region}.jsonl', 'w') as outfile:
        for entry in ranked_matches:
            json.dump(entry, outfile)
            outfile.write('\n')
    del ranked_matches
    
    
    for i, timeline in enumerate(timeline_list):
        if i in is_ranked:
            ranked_timelines.append(timeline)
    del timeline_list
            
    

    with open(f'../data/cleaned/timelines_{region}.jsonl', 'w') as outfile:
        for entry in ranked_timelines:
            json.dump(entry, outfile)
            outfile.write('\n')   
    del ranked_timelines
    del is_ranked
    del game_ids

In [9]:
def remove_short_games(region):
    match_list = []
    timeline_list = []
    
    with jsonlines.open(f'../data/cleaned/matches_{region}.jsonl') as infile:
                for line in infile.iter():
                    match_list.append(line) 
    del infile
                 
    with jsonlines.open(f'../data/cleaned/timelines_{region}.jsonl') as infile:
                for line in infile.iter():
                    timeline_list.append(line) 
    del infile
                    
    good_matches = []
    good_timelines = []
    valid = []
    game_ids = set()
    
    for i, match in enumerate(match_list):
        if match['gameDuration'] / 60 > 15 and match['gameId'] not in game_ids:
            add = True
            # Remove games where a player didn't reach level 6 (based on EDA)
            for participant in match['participants']:
                if participant['stats']['champLevel'] < 7:
                    add = False
            if add:
                good_matches.append(match)
                valid.append(i)
                game_ids.add(match['gameId'])
            else:
                pass
    
    del match_list
  
    with open(f'../data/cleaned/matches_{region}.jsonl', 'w') as outfile:
        for entry in good_matches:
            json.dump(entry, outfile)
            outfile.write('\n')
    del good_matches
    
    for i, timeline in enumerate(timeline_list):
        if i in valid:
            good_timelines.append(timeline)
    
    del timeline_list
            
    

    with open(f'../data/cleaned/timelines_{region}.jsonl', 'w') as outfile:
        for entry in good_timelines:
            json.dump(entry, outfile)
            outfile.write('\n')
            
    del good_timelines
    del valid
    del game_ids

Pulling all Challenger players' match histories:

In [ ]:
all_challengers = get_all_challengers()

all_challengers.reset_index(drop=True,inplace=True)

In [ ]:
match_hist_list = []
with concurrent.futures.ThreadPoolExecutor(max_workers = 11) as executor:
    for result in executor.map(get_challenger_match_hists, region_base_url_dict.keys()):
        match_hist_list.append(result)

In [ ]:
challenger_match_hists = pd.concat(match_hist_list)

In [ ]:
challenger_match_hists.reset_index(drop=True,inplace=True)

In [ ]:
challenger_match_hists.to_csv('../data/challenger_match_hists_ranked_only.csv', index=False)

Downloading match and timeline data for all challenger match histories:

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers = 11) as executor:
    executor.map(scrape_seeds, region_base_url_dict.keys())

And finally removing all duplicate games and games which lasted less than 15 minutes:

In [ ]:
for region in region_base_url_dict.keys():
    remove_duplicates(region)

In [ ]:
for region in region_base_url_dict.keys():
    remove_short_games(region)

## Data Processing

In [23]:
def get_stats(summoner, region, matches, timelines, team_participants = ([1,2,3,4,5],[6,7,8,9,10])):
# For each player:
    output = pd.DataFrame(columns=['champ_played', 'role','lane', 'csd_10',
                                  'gold_d_10','xpd_10','dmg_share','dmg_taken_share','vision_score',
                                  'kill_participation','obj_dmg_share','dragons','barons','wards_cleared',
                                  'vision_wards_purchased','kda_early', 'kda_mid', 'kda_late', 
                                   'solo_kills', 'teamfight_kills', 'skirmish_kills', 'wards_early', 
                                   'wards_mid', 'wards_late'])
    # Main role = most played role where role is played > 40% of the time
    # Main champion = most played champion
    # Select all matches and timelines that they are in
    for i, match in enumerate(matches):
        is_in_game = summoner in [summoner['player']['summonerId'] for summoner in match['participantIdentities']]
        if is_in_game:
        # For each match:
        # Match info:
            game_id = match['gameId']
            player_id = int([player['participantId'] for player in match['participantIdentities'] if player['player']['summonerId'] == summoner][0]) - 1
            player_name = match['participantIdentities'][player_id]['player']['summonerName']
            
            # Identify champion played
            champ_played = match['participants'][player_id]['championId']
            
            # Identify role played
            role = match['participants'][player_id]['timeline']['role']
            lane = match['participants'][player_id]['timeline']['lane']
            # Identify lane opponent
            team = match['participants'][player_id]['teamId']
            team_index = int(str(team)[0]) - 1
            teammates = team_participants[team_index]
               
            # Get CSD, GoldD and XPD @ 10
            csd_10 = match['participants'][player_id]['timeline']['creepsPerMinDeltas']['0-10']
            gold_d_10 = match['participants'][player_id]['timeline']['goldPerMinDeltas']['0-10']
            xpd_10 = match['participants'][player_id]['timeline']['xpPerMinDeltas']['0-10']
            
            # Get DMG share
            total_dmg = np.sum([player['stats']['totalDamageDealtToChampions'] for player in match['participants'] if player['teamId'] == team])
            dmg_share = match['participants'][player_id]['stats']['totalDamageDealtToChampions'] / total_dmg
            
            # Get DMG Taken share
            total_dmg_taken = np.sum([player['stats']['totalDamageTaken'] for player in match['participants'] if player['teamId'] == team])
            dmg_taken_share = match['participants'][player_id]['stats']['totalDamageTaken'] / total_dmg_taken
            
            # Get vision score
            vision_score = match['participants'][player_id]['stats']['visionScore']
            
            # Get overall kill participation
            team_kills = np.sum([player['stats']['kills'] for player in match['participants'] if player['participantId'] in teammates]) + 1
            kill_participation = (match['participants'][player_id]['stats']['kills'] + match['participants'][player_id]['stats']['assists']) / team_kills
            
            # % of team's objective damage % of team's turret damage
            total_obj_dmg = np.sum([player['stats']['damageDealtToObjectives'] for player in match['participants'] if player['teamId'] == team])
            obj_dmg_share = match['participants'][player_id]['stats']['damageDealtToObjectives'] / total_obj_dmg
            
            # Team % of dragons killed
            team_dragons = match['teams'][team_index]['dragonKills']
            total_dragons = team_dragons + match['teams'][0 if team_index == 1 else 1]['dragonKills']
            dragons = team_dragons / (total_dragons + 1)
            
            # Team % of barons killed
            team_barons = match['teams'][team_index]['baronKills']
            total_barons = team_barons + match['teams'][0 if team_index == 1 else 1]['baronKills']
            barons = team_barons / (total_barons + 1)
            
            # Get wards cleared
            wards_cleared = match['participants'][player_id]['stats']['wardsKilled']
            
            # Get pinks purchased
            vision_wards_purchased = match['participants'][player_id]['stats']['visionWardsBoughtInGame']
            
            # For each timeline:
            timeline = timelines[i]
            
            kills = 0
            kda = 0
            kda_early = 0
            kda_mid = 0
            kda_late = 0
            solo_kills = 0
            teamfight_kills = 0
            skirmish_kills = 0
            wards = 0
            wards_early = 0
            wards_mid = 0
            wards_late = 0
            
            for i, frame in enumerate(timeline['frames']):
                for event in frame['events']:
                    # Get 0-10 K+D+A
                    # Get 10-20 K+D+A
                    # Get 20+ K+D+A
                    if event['type'] == 'CHAMPION_KILL':
                        if (player_id + 1) == event['killerId'] or (player_id + 1) == event['victimId'] or (player_id + 1) in event['assistingParticipantIds']:
                            kda += 1
                            if i < 12:
                                kda_early += 1
                            elif i < 22:
                                kda_mid += 1
                            else:
                                kda_late += 1

                        if (player_id + 1) == event['killerId']:
                            kills += 1
                    # get number of solo kills
                    # get number of skirmish kills
                    # get number of teamfight kills
                            if event['assistingParticipantIds'] == []:
                                solo_kills += 1
                            if len(event['assistingParticipantIds']) == 1:
                                skirmish_kills += 1
                            if  len(event['assistingParticipantIds']) > 1:
                                teamfight_kills += 1
                    # Get 0-10 wards placed
                    # Get 10-20 wards placed
                    # Get 20+ wards placed
                    if event['type'] == 'WARD_PLACED' and (player_id + 1) == event['creatorId']:
                        wards += 1
                        if i < 12:
                            wards_early += 1
                        elif i < 22:
                            wards_mid += 1
                        else:
                            wards_late += 1 
            
            solo_kills /= kills + 1
            skirmish_kills /= kills + 1
            teamfight_kills /= kills + 1
            wards_early /= wards + 1
            wards_mid /= wards + 1
            wards_late /= wards + 1 
            kda_early /= kda + 1
            kda_mid /= kda + 1
            kda_late /= kda + 1
            
            # Wards cleared and vision wards purchased as a % of wards placed
            wards_cleared /= wards + 1
            vision_wards_purchased /= wards + 1
            
            output = output.append({'champ_played':champ_played, 'role':role, 'lane':lane,
             'csd_10':csd_10,'gold_d_10':gold_d_10,'xpd_10':xpd_10,'dmg_share':dmg_share,
             'dmg_taken_share':dmg_taken_share,'vision_score':vision_score,
             'kill_participation':kill_participation,'obj_dmg_share':obj_dmg_share,'dragons':dragons,
             'barons':barons,'wards_cleared':wards_cleared,'vision_wards_purchased':vision_wards_purchased,
             'kda_early':kda_early, 'kda_mid':kda_mid, 'kda_late':kda_late, 'solo_kills':solo_kills, 
             'teamfight_kills':teamfight_kills, 'skirmish_kills':skirmish_kills, 'wards_early':wards_early, 
             'wards_mid':wards_mid, 'wards_late':wards_late}, ignore_index=True)
    
    most_played_champ = output['champ_played'].value_counts().index[0]
    most_played_role = output['role'].value_counts().index[0]
    most_played_lane = output['lane'].value_counts().index[0]
    
    output.drop(columns=['champ_played','role','lane'], inplace=True)

    output = pd.DataFrame(output.mean()).transpose()
    output['summoner_id'] = [summoner]
    output['most_played_champ'] = most_played_champ
    output['most_played_role'] = most_played_role
    output['most_played_lane'] = most_played_lane
    output['region'] = region
    output['player_name'] = player_name
    return output

## Recommender

## Evaluation and User Feedback
"Personally, I found the champion stuff the most interesting part of the data, especially identifying trends across multiple players. I feel like the whole thing gave me some interesting ideas and allowed me to look at the way I play in a different way." 

## Future Work

## References
- [Riot Games API](https://developer.riotgames.com/apis)
- [How Well does Personalized Marketing Work?](https://knowledge.wharton.upenn.edu/article/recommended-for-you-how-well-does-personalized-marketing-work/)
- [Recommender Systems in Python 101](https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101)
- [Building the Next New York Times Recommendation Engine](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine)
- [The Netflix Recommender System: Algorithms, Business Value, and Innovation](https://dl.acm.org/doi/10.1145/2843948)
- [How Machine Learning Fuels Your Netflix Addiction](https://www.rtinsights.com/netflix-recommendations-machine-learning-algorithms/)
- [The Treasure Chest of League of Legends: Riot’s APIs](https://medium.com/@montane/the-treasure-chest-of-league-of-legends-riots-apis-64816f4d3f84)
- [Part 1 of Riot API: Data Downpour](https://towardsdatascience.com/data-downpour-b1c4b41d7862)